In [1]:
datafolder = './'
filename = datafolder + 'TGSCData.json'

In [2]:
from urllib3 import PoolManager
from lxml import etree
#http = PoolManager()

def get_source_page(url):
    http = PoolManager()
    response = http.request('GET',url)
    source_page = response.read()
    return source_page

In [3]:
url = 'http://www.thegoodscentscompany.com/fragonly-a.html'
http = PoolManager()
parser = etree.HTMLParser()
alpha_urls = []

with http.urlopen('GET',url, preload_content=False) as f:
     tree = etree.parse(f, parser)

for x in tree.getroot().xpath("//div[@class='alphaENIC1']"):#.find_class('alphaENIC1')
    for y in x.getiterator():
        if y.attrib.has_key('href'):
            alpha_urls.append(y.attrib.get('href'))
            

In [6]:
from collections import OrderedDict
alpha_urls = list(OrderedDict.fromkeys(alpha_urls))

In [7]:
alpha_urls

['http://www.thegoodscentscompany.com/fragonly-a.html',
 'http://www.thegoodscentscompany.com/fragonly-b.html',
 'http://www.thegoodscentscompany.com/fragonly-c.html',
 'http://www.thegoodscentscompany.com/fragonly-d.html',
 'http://www.thegoodscentscompany.com/fragonly-e.html',
 'http://www.thegoodscentscompany.com/fragonly-f.html',
 'http://www.thegoodscentscompany.com/fragonly-g.html',
 'http://www.thegoodscentscompany.com/fragonly-h.html',
 'http://www.thegoodscentscompany.com/fragonly-i.html',
 'http://www.thegoodscentscompany.com/fragonly-jk.html',
 'http://www.thegoodscentscompany.com/fragonly-l.html',
 'http://www.thegoodscentscompany.com/fragonly-m.html',
 'http://www.thegoodscentscompany.com/fragonly-n.html',
 'http://www.thegoodscentscompany.com/fragonly-o.html',
 'http://www.thegoodscentscompany.com/fragonly-p.html',
 'http://www.thegoodscentscompany.com/fragonly-q.html',
 'http://www.thegoodscentscompany.com/fragonly-r.html',
 'http://www.thegoodscentscompany.com/fragonly-

In [8]:
def get_xhtml(url,pool):
    with pool.urlopen('GET',url, preload_content=False) as f:
        tree = etree.parse(f, parser)
    return tree

In [9]:
def clean_string(txt):
    txt = txt.replace('\xa0',' ').replace('Â\xa0',' ').replace('Â','')
    return txt

In [10]:
def get_sub_before_first_comma(txt):
        txt = txt + ','
        l = txt[0:txt.find(',')].strip()
        try:
            l.remove("")
        except: None
        return l

In [11]:
def split_html_string(txt):
    odor_descr_str = txt.replace('Â\xa0Â\xa0',',').replace('Â\xa0Â','').replace('Â','')
    odor_descr_list = odor_descr_str.split(',')
    try:
        odor_descr_list.remove("")
    except ValueError: None
    return odor_descr_list
    

In [12]:
def get_substantivity(txt):
    txt = txt[0:txt.lower().find('hour(s)')].strip()
    return txt

In [13]:
import re
def get_numeric_value(txt):
    txt = re.findall(r'[0-9.]',txt)
    txt = ''.join(txt)
    return ''.join(txt)
    

In [14]:
def get_filename(url,folder):
    return folder + '/' + url[url.rfind('/')+1:url.rfind('.')] + '.json'

In [15]:
def get_min_value(txt):
    txt = txt.replace('Â','')
    txt = txt.split(' ')
    txt = txt[0].replace(' to','').replace('>','').replace('<','')
    return txt  

In [16]:
def get_flash_point_celsium(txt):
    txt = txt.split('Â')
    txt = txt[4].replace('(','').replace(' ','').replace('>','').replace('<','')
    return txt
    

In [17]:
def get_logP(txt):
    txt = txt.replace('(est)','').replace(' ','')
    return txt

In [18]:
import pandas as pd
import unidecode
import json
import numpy as np

http = PoolManager()
tgsc_component = {}

for url in alpha_urls:
    print(url)

    #tgsc_component = {}
    xhtml = get_xhtml(url,http)
    component_table = xhtml.xpath("/html//table[1]")[0]
    for component in component_table.getiterator():
        #print('    ' + str(component))
       
        if component.attrib.has_key('href'):
            #print('    ' + component.values()[0])
            c_url = component.values()[0]
            c_xhtml = get_xhtml(c_url,http)
            component_dict = {}
            name = c_xhtml.find(".//title").text
            component_dict['name'] = clean_string(name) #.replace('\xa0',' ').replace('Â\xa0',' ')
            for section in c_xhtml.xpath('//table[@id="chemdata"]'):
                #print(section)
                df_main = pd.DataFrame()
                
                try:
                    df_main = pd.read_html(io=etree.tostring(section, encoding='utf-8'))[0]
                except: 
                    print('    ' + c_url)
                    raise Exception("Convert to Pandas Error")
                    
                df_main.columns = ['property', 'value']
                try:
                    iupac = df_main[df_main["property"] == "IUPAC Name:"]["value"].values[0]
                    component_dict['iupac'] = clean_string(iupac) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
                try:
                    full_name = df_main[df_main["property"] == "Name:"]["value"].values[0]
                    component_dict['full_name'] = clean_string(full_name) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
                try:
                    cas = df_main[df_main["property"] == "CAS Number:"]["value"].values[0]
                    component_dict['cas'] = clean_string(cas) #.replace('\xa0',' ').replace('Â\xa0',' ')
                except IndexError: None
            sectionclass = c_xhtml.xpath("//div[@class='sectionclass']")
            for s in sectionclass:
                #print(s.text)
                if s.text == "Physical Properties: ":
                    #print(s.getnext())
                    df_phy = pd.DataFrame()
                    df_phy = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                    if len(df_phy) > 1:
                        df_phy.columns = ['property', 'value']
                        try:
                            specific_gravity = df_phy[df_phy["property"] == "Specific Gravity:"]["value"].values[0]
                            #print("specific_gravity=" + specific_gravity)
                            #print("specific_gravity=" + get_min_value(specific_gravity))
                            component_dict['specific_gravity'] = get_min_value(clean_string(specific_gravity)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            boiling_point = df_phy[df_phy["property"] == "Boiling Point:"]["value"].values[0]
                            #print("boiling_point=" + boiling_point)
                            #print("boiling_point=" + get_min_value(clean_string(boiling_point)))
                            component_dict['boiling_point'] = get_min_value(clean_string(boiling_point)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            refractive_index = df_phy[df_phy["property"] == "Refractive Index:"]["value"].values[0]
                            #print("refractive_index=" + refractive_index)
                            #print("refractive_index=" + get_min_value(refractive_index))
                            component_dict['refractive_index'] = get_min_value(clean_string(refractive_index)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            flash_point = df_phy[df_phy["property"] == "Flash Point:"]["value"].values[0]
                            #print("flash_point=" + flash_point)
                            #print("flash_point=" + get_numeric_value(get_flash_point_celsium(flash_point)))
                            if str(flash_point) != 'nan':
                                component_dict['flash_point'] = get_numeric_value(get_flash_point_celsium(flash_point)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            logP = df_phy[df_phy["property"] == "logP (o/w):"]["value"].values[0]
                            #print("logP=" + logP)
                            #print("logP=" + get_logP(logP))
                            component_dict['logP'] = get_logP(clean_string(logP)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
    
                if s.text == "Organoleptic Properties: ":
                    #print(s.getnext())
                    df_org = pd.DataFrame()
                    df_org = pd.read_html(io=etree.tostring(s.getnext(), encoding='utf-8'))[0]
                    if len(df_org) > 1:
                        df_org.columns = ['property', 'value']
                        try:
                            odor_type = df_org[df_org["property"] == "Odor Type:"]["value"].values[0]
                            #print(odor_type)
                            component_dict['odor_type'] = clean_string(odor_type) #.replace('\xa0',' ').replace('Â\xa0',' ')
                        except IndexError: None
                        try:
                            #odor_type = df[df["property"] == "Odor Type:"]["value"].values[0]
                            odor_strength = df_org[df_org["property"] == "Odor Strength:"]["value"].values[0]
                            #print(odor_strength)
                            #print(clean_string(get_sub_before_first_comma(odor_strength)))
                            component_dict['odor_strength'] = clean_string(get_sub_before_first_comma(odor_strength)) #.replace('\xa0',' ').replace('Â\xa0',' ')
                            #component_dict['odor_strength'] = odor_strength
                        except IndexError: None
                        try:
                            odor_description = split_html_string(df_org['value'][2])
                            if len(odor_description) > 0:
                                component_dict['odor_description'] = odor_description
                        except IndexError: None
                        except KeyError: print('    ' + c_url)
                        try:
                            odor_substantivity = df_org[df_org["property"] == "Substantivity:"]["value"].values[0]
                            try:
                                odor_substantivity = get_numeric_value(clean_string(odor_substantivity))
                                component_dict['odor_substantivity'] = get_substantivity(odor_substantivity)

                            except AttributeError: None
                        except IndexError: None
            tgsc_component[c_url] = component_dict
            #filename = get_filename(url,datafolder)

with open(filename, 'w') as outfile:
    json.dump(tgsc_component, outfile)


http://www.thegoodscentscompany.com/fragonly-a.html
    http://www.thegoodscentscompany.com/data/co1067571.html
    http://www.thegoodscentscompany.com/data/rw1537431.html
    http://www.thegoodscentscompany.com/data/rw1003241.html
    http://www.thegoodscentscompany.com/data/rw1004091.html
    http://www.thegoodscentscompany.com/data/es1052621.html
    http://www.thegoodscentscompany.com/data/ab1657301.html
    http://www.thegoodscentscompany.com/data/co1657731.html
    http://www.thegoodscentscompany.com/data/rw1056571.html
    http://www.thegoodscentscompany.com/data/ab1092781.html
http://www.thegoodscentscompany.com/fragonly-b.html
    http://www.thegoodscentscompany.com/data/rw1464061.html
    http://www.thegoodscentscompany.com/data/rw1464271.html
    http://www.thegoodscentscompany.com/data/rw1412231.html
    http://www.thegoodscentscompany.com/data/rw1122951.html
    http://www.thegoodscentscompany.com/data/rw1123231.html
    http://www.thegoodscentscompany.com/data/ex1022771.h

In [ ]:
df_phy.head(10)

In [ ]:
tgsc_component